In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import torch
from functools import partial
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import torch.nn as nn
from embedding import CustomEmbedding
from transformer import EncoderDecoderTransformer
from utils import padding_collate_fn
import numpy as np

from generate_data import RandomIntegerDataset

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
n_real_tokens = 10
PAD_TOKEN_IDX = n_real_tokens
SOS_TOKEN_IDX = n_real_tokens + 1
EOS_TOKEN_IDX = n_real_tokens + 2
vocab_size = n_real_tokens + 3
D_MODEL = 64

embeddings = CustomEmbedding(vocab_size, d_model = D_MODEL) # 3 = PAD, SOS, EOS

indices = torch.tensor([1,9])

# print(embeddings.embeddings.weight)
print(embeddings.embeddings(indices))

tensor([[ 2.5912, -1.4263, -0.4131, -0.4404, -0.6530, -0.0486, -0.5205,  0.0795,
         -0.3692,  1.3514, -1.7537,  1.6198, -0.6558, -0.6826, -0.1505,  0.1238,
          1.8447, -0.0927,  0.6445,  0.1265, -0.6074, -1.6156, -1.3574,  0.2402,
          1.1803,  1.2475,  0.0982,  0.6528, -1.1838,  1.1011, -0.7063, -1.6845,
         -0.0553,  0.0330,  1.1433, -0.2270,  1.4425, -1.8413,  1.0542, -0.3801,
          0.2648, -2.4542,  0.0830,  1.9601,  0.2750, -0.5426,  1.0269,  1.7440,
         -2.4217,  0.3061,  0.5601, -1.4078,  0.3974,  0.2823, -1.2763, -1.0821,
         -0.0217,  0.6498,  0.8835,  0.3206, -0.0731, -0.1787, -1.3928,  1.1576],
        [-0.0983,  2.7286,  1.7257, -0.4248, -1.6255, -1.6202,  0.8901,  0.6736,
         -1.3352,  0.2541, -1.3306, -0.9637,  0.1807, -0.7534, -0.6530,  1.9191,
         -0.9400, -0.5747, -1.5215,  0.5857, -1.7653,  1.7890, -0.1325, -0.0090,
         -2.3319,  1.8486, -1.4751,  0.8222, -0.7151, -1.6365,  0.3117,  1.3396,
         -0.6446,  0.7574, 

In [4]:
MAX_CONTEXT_WINDOW = 50

BATCH_SIZE = 64
MIN_SEQ_LEN = 2
MAX_SEQ_LEN = min(20, MAX_CONTEXT_WINDOW)

NUM_TRAINING_SEQUENCES = 10000
NUM_VALIDATION_SEQUENCES = 1000

VOCAB = [i for i in range(n_real_tokens)] # does not include SOS, EOS, PAD

VOCAB_MAP = dict()

for i, token in enumerate(VOCAB):
    VOCAB_MAP[i] = token
VOCAB_MAP[len(VOCAB_MAP)] = '<PAD>'
VOCAB_MAP[len(VOCAB_MAP) + 1] = '<SOS>'
VOCAB_MAP[len(VOCAB_MAP) + 2] = '<EOS>'

train_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, NUM_TRAINING_SEQUENCES, VOCAB)
train_dataloader = DataLoader(train_rand_ds, batch_size = BATCH_SIZE, shuffle = True, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

val_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, NUM_VALIDATION_SEQUENCES, VOCAB)
val_dataloader = DataLoader(val_rand_ds, batch_size = BATCH_SIZE, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

In [5]:
input, label = next(iter(train_dataloader))
print(input[0])
print(input[1])
print(label)

tensor([[ 4,  4,  0,  ..., 10, 10, 10],
        [ 9,  4,  3,  ..., 10, 10, 10],
        [ 4,  5,  0,  ...,  9,  5,  1],
        ...,
        [ 6,  5,  6,  ..., 10, 10, 10],
        [ 7,  0,  5,  ..., 10, 10, 10],
        [ 4,  4,  1,  ..., 10, 10, 10]])
tensor([[11,  0,  0,  ..., 10, 10, 10],
        [11,  3,  3,  ..., 10, 10, 10],
        [11,  0,  0,  ...,  9,  9,  9],
        ...,
        [11,  0,  1,  ..., 10, 10, 10],
        [11,  0,  0,  ..., 10, 10, 10],
        [11,  0,  1,  ..., 10, 10, 10]])
tensor([[ 0,  0,  1,  ..., 10, 10, 10],
        [ 3,  3,  4,  ..., 10, 10, 10],
        [ 0,  0,  0,  ...,  9,  9, 12],
        ...,
        [ 0,  1,  2,  ..., 10, 10, 10],
        [ 0,  0,  2,  ..., 10, 10, 10],
        [ 0,  1,  4,  ..., 10, 10, 10]])


In [6]:
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_IDX, reduction = 'sum')

model = EncoderDecoderTransformer(
                    embeddings = embeddings, 
                    vocab_size = vocab_size, 
                    d_model = D_MODEL, 
                    num_attention_heads = 4, 
                    num_encoder_layers = 2, 
                    num_decoder_layers = 2, 
                    dim_feedforward = 32, 
                    dropout = 0.0,
                    max_context_window = MAX_CONTEXT_WINDOW,
                    use_pre_lnorm = True)

optim = torch.optim.SGD(params = model.parameters(), lr = 1e-4, momentum = 0.9, weight_decay = 1e-4)

/Users/paul/opt/miniconda3/envs/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def greedy_decode(source: torch.Tensor, model: nn.Module) -> torch.Tensor:
    """
    Designed to do autoregressive inference in an Encoder-Decoder transformer.

    This greedy decoder always predicts the vocabulary token corresponding to the highest logit.

    Takes the source sequence and the Encoder-Decoder to produce a predicted
    sequence starting from <SOS>.

    Note: This function *can* handle batches of sequences.

    Args:
        source - The source sequence to be passed to the Transformer's encoder block.
        model - The Encoder-Decoder transformer over which to greedy decode

    Returns:
        target - The batch of predicted sequences corresponding to the input sources.
        target_logits - (# batch elements =) batch_size (# rows =) seq_len (# cols =) vocab-dimensional vectors, each of which
        corresponds to the set of logits on a particular inference step within a given sequence.
    """
    batch_size = source.size(dim = 0)

    encoder_output, source_pad_mask = model.encode(source)

    # target will contain num_batch sequences of indices that are the predicted next-words for each batch element
    target = torch.full((batch_size, 1), SOS_TOKEN_IDX) # target.shape: [batch_size, num_loops_complete - 1]
    target_logits = torch.zeros((batch_size, 1, vocab_size))

    finished = torch.full((batch_size, ), False)

    while not finished.all() and target.size(dim = 1) <= MAX_CONTEXT_WINDOW:

        decoder_output, _ = model.decode(target, encoder_output, source_pad_mask)
        pred_logits = model.project_into_vocab(decoder_output) # pred_logits.shape: [batch_size, seq_len, vocab_size]

        last_row_pred_logits = pred_logits[:, -1, :] # last_row_pred_logits.shape == [batch_size, vocab_size]

        # Track next-word logits for loss_fn later.
        target_logits = torch.concat((target_logits, last_row_pred_logits.unsqueeze(1)), dim = 1)

        predictions = torch.argmax(last_row_pred_logits, dim = -1) # predictions.shape: [batch_size]

        # For any finished sequences (i.e. previous EOS-producers), force their prediction from this round to be a pad.
        predictions[finished] = PAD_TOKEN_IDX

        # Mark any additional sequences that just produced an EOS as finished.
        finished |= predictions == EOS_TOKEN_IDX

        target = torch.concat((target, predictions.reshape(-1, 1)), dim = 1) # target.shape: [batch_size, num_loops_complete]

    return target, target_logits[:, 1:, :]

In [8]:
def run_train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """
    Runs one training epoch (processing the entire training dataset once).
    
    Uses Teacher Forcing to train token-to-token mapping quality without cascading errors and for parallelization.

    Args:
        dataloader - The dataloader to process the dataset in BATCH_SIZE batches
        model - The Encoder-Decoder that is being trained
        loss_fn - The loss function to calculate the model's correctness
        optimizer - The optimizer to improve the model's weights
        calculate_sequence_accuracy - A flag to mark whether sequence-level correctness should be tracked
        calculate_token_accuracy - A flag to mark whether token-level correctness should be tracked
    """
    model.train()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    for (source, target), label in tqdm(dataloader):

        # FORWARD
        pred_logits = model(source, target)

        # pred_logits.shape: [batch_size, seq_len, vocab_size]
        # label.shape: [batch_size, seq_len]

        # CrossEntropyLoss (loss_fn) only takes 2D predictions (n_batch * seq_len, vocab_size) and 1D labels (n_batch * seq_len)
        batch_loss = loss_fn(pred_logits.view(-1, pred_logits.size(-1)), label.view(-1))

        # LOG
        with torch.no_grad():
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences.item()

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens.item()

                num_tokens += torch.numel(label)

        # BACKWARD
        batch_loss.backward()

        # OPTIMIZE
        optimizer.step()
        optimizer.zero_grad()

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_gold_validation_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """
    Runs one validation epoch (processing the entire validation dataset once). 

    Uses Teacher Forcing (i.e. "gold") to evaluate token-to-token mapping quality and for parallelization.

    Args:
        dataloader - The dataloader to process the dataset in BATCH_SIZE batches
        model - The Encoder-Decoder that is being trained
        loss_fn - The loss function to calculate the model's correctness
        calculate_sequence_accuracy - A flag to mark whether sequence-level correctness should be tracked
        calculate_token_accuracy - A flag to mark whether token-level correctness should be tracked
    """
    model.eval()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    with torch.no_grad():
        
        for (source, target), label in tqdm(dataloader):
            
            # FORWARD
            pred_logits = model(source, target)
            batch_loss = loss_fn(pred_logits.view(-1, pred_logits.size(-1)), label.view(-1))

            # LOG
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences.item()

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens.item()

                num_tokens += torch.numel(label)

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_autoregressive_validation_loop(dataloader: DataLoader, model: nn.Module):
    """
    Runs one autoregressive validation epoch (processing the entire validation dataset once). 

    Args:
        dataloader - The dataloader to process the dataset in BATCH_SIZE batches
        model - The Encoder-Decoder that is being trained
    """
    model.eval()

    correct_sequences = 0
    incorrect_sequences = 0
    total_sequences = 0

    with torch.no_grad():
        
        for (source, _), label in tqdm(dataloader):

            # FORWARD
            pred_indices, pred_logits = greedy_decode(source, model)

            np_source_indices = source.numpy().copy()
            np_pred_target_indices = pred_indices.numpy().copy()

            token_values = np.array(list(VOCAB_MAP.values()))
            predicted_source_tokens = token_values[np_source_indices]
            predicted_target_tokens = token_values[np_pred_target_indices]

            for s, t in zip(predicted_source_tokens, predicted_target_tokens):
                source_end_index = np.argmax(s == '<PAD>')  if '<PAD>' in s else len(s)
                target_end_index = np.argmax(t == '<EOS>')
                if np.array_equal(np.sort(s[:source_end_index]), t[1:target_end_index]):
                    correct_sequences += 1
                else:
                    incorrect_sequences += 1
                    print(f'Incorrect Sequence {incorrect_sequences}:')
                    print(np.sort(s[:source_end_index]))
                    print(t[1:target_end_index])
                    print(f'{'Source:':<20} {s}\n{'Predicted Target:':<20} {t}', end = '\n\n')

            total_sequences += predicted_target_tokens.shape[0]

    return correct_sequences / total_sequences


In [9]:
EPOCHS = 10

training_losses = list()
training_sequence_accuracies = list()
training_token_accuracies = list()

gold_validation_losses = list()
gold_validation_sequence_accuracies = list()
gold_validation_token_accuracies = list()

for i in range(EPOCHS):
    # print(f'Running epoch {i+1}...')

    training_loss, training_sequence_accuracy, training_token_accuracy = run_train_epoch(train_dataloader, model, loss_fn, optim, calculate_sequence_accuracy = True, calculate_token_accuracy = True)

    training_losses.append(training_loss)
    training_sequence_accuracies.append(training_sequence_accuracy)
    training_token_accuracies.append(training_token_accuracy)

    gold_val_loss, gold_val_sequence_accuracy, gold_val_token_accuracy = run_gold_validation_loop(val_dataloader, model, loss_fn, calculate_sequence_accuracy = True, calculate_token_accuracy = True)
    
    gold_validation_losses.append(gold_val_loss)
    gold_validation_sequence_accuracies.append(gold_val_sequence_accuracy)
    gold_validation_token_accuracies.append(gold_val_token_accuracy)

print(training_losses)
print(training_sequence_accuracies)
print(training_token_accuracies)

print()

print(gold_validation_losses)
print(gold_validation_sequence_accuracies)
print(gold_validation_token_accuracies)

100%|██████████| 16/16 [00:00<00:00, 56.11it/s]

[5.9956224903106685, 1.8113396434783935, 0.9595275775909424, 0.5739403173804283, 0.5067678166747093, 0.8473928666114807, 0.08300657985210419, 0.03322321484088898, 0.7289299174726009, 0.05126665424108505]
[0.0028, 0.0159, 0.0256, 0.0348, 0.0365, 0.0317, 0.0515, 0.0541, 0.037, 0.0533]
[0.4620571504924792, 0.5423993058204287, 0.5557701846764347, 0.5625143129770992, 0.5633957840969734, 0.5578385952871197, 0.5699454697986577, 0.5706679389312977, 0.5614829245931698, 0.5704603556166056]

[9.56902621459961, 0.8637341880798339, 3.5278837890625, 2.544419677734375, 0.14738343524932862, 0.30951369285583497, 0.038920241713523866, 0.03802552580833435, 0.12167732405662536, 0.0335579354763031]
[0.0, 0.021, 0.0, 0.0, 0.039, 0.045, 0.049, 0.048, 0.047, 0.049]
[0.4338937714940772, 0.5480989682842949, 0.5136606801681315, 0.5250286587695835, 0.559849063813527, 0.5585594191822698, 0.5611864730607566, 0.5610909438288116, 0.5605177684371417, 0.5611864730607566]


In [10]:
acc = run_autoregressive_validation_loop(val_dataloader, model)
print(acc)

 12%|█▎        | 2/16 [00:00<00:05,  2.57it/s]

Incorrect Sequence 1:
['0' '0' '0' '0' '0' '0' '0' '2' '3' '3' '7' '7' '7' '9' '9' '9' '9' '9'
 '9' '9']
[]
Source:              ['9' '3' '0' '0' '2' '9' '9' '9' '7' '0' '9' '9' '7' '7' '9' '0' '0' '0'
 '0' '3']
Predicted Target:    ['<SOS>' '0' '0' '0' '0' '0' '0' '0' '2' '3' '3' '7' '7' '7' '9' '9' '9'
 '9' '9' '9' '0' '0' '0' '0' '0' '0' '0' '0' '0' '7' '7' '7' '7' '7' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0']

Incorrect Sequence 2:
['1' '1' '3' '3' '3' '3' '3' '3' '4' '5' '7' '9' '9' '9' '9' '9']
['1' '1' '3' '3' '3' '3' '3' '3' '4' '5' '7' '9' '9' '9' '9']
Source:              ['9' '1' '9' '3' '3' '3' '9' '1' '9' '3' '3' '5' '3' '9' '7' '4' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '1' '1' '3' '3' '3' '3' '3' '3' '4' '5' '7' '9' '9' '9' '9'
 '<EOS>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']



 38%|███▊      | 6/16 [00:01<00:02,  4.92it/s]

Incorrect Sequence 3:
['0' '0' '0' '0' '0' '0' '3' '3' '3' '4' '4' '4' '5' '6' '7' '7' '8' '9'
 '9']
['0' '0' '0' '0' '0' '3' '3' '3' '4' '4' '4' '5' '6' '7' '7' '8' '9' '9']
Source:              ['3' '4' '8' '0' '0' '5' '9' '0' '0' '4' '0' '0' '9' '6' '3' '7' '4' '3'
 '7' '<PAD>']
Predicted Target:    ['<SOS>' '0' '0' '0' '0' '0' '3' '3' '3' '4' '4' '4' '5' '6' '7' '7' '8'
 '9' '9' '<EOS>' '<PAD>' '<PAD>']



 50%|█████     | 8/16 [00:01<00:01,  5.51it/s]

Incorrect Sequence 4:
['0' '1' '1' '1' '3' '4' '4' '4' '5' '5' '5' '5' '7' '7' '7' '7' '7' '8'
 '8']
['0' '1' '1' '1' '3' '4' '4' '4' '5' '5' '5' '5' '7' '7' '7' '7' '8' '8']
Source:              ['1' '5' '4' '1' '4' '0' '3' '7' '1' '7' '5' '7' '7' '7' '8' '8' '4' '5'
 '5' '<PAD>']
Predicted Target:    ['<SOS>' '0' '1' '1' '1' '3' '4' '4' '4' '5' '5' '5' '5' '7' '7' '7' '7'
 '8' '8' '<EOS>' '<PAD>' '<PAD>']

Incorrect Sequence 5:
['1' '1' '2' '3' '5' '5' '5' '5' '5' '6' '6' '6' '9' '9']
['1' '1' '2' '3' '5' '5' '5' '5' '6' '6' '6' '9' '9']
Source:              ['5' '2' '5' '5' '9' '5' '9' '1' '1' '6' '5' '6' '6' '3' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '1' '1' '2' '3' '5' '5' '5' '5' '6' '6' '6' '9' '9' '<EOS>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']



 81%|████████▏ | 13/16 [00:02<00:00,  6.04it/s]

Incorrect Sequence 6:
['2' '7' '7' '8' '8' '8' '8' '9' '9' '9']
['2' '7' '8' '8' '8' '9' '9']
Source:              ['9' '9' '8' '7' '2' '9' '8' '8' '7' '8' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '2' '7' '8' '8' '8' '9' '9' '<EOS>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>']



100%|██████████| 16/16 [00:03<00:00,  5.10it/s]

0.994
